In [ ]:
!pip install saliency

In [ ]:
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import average_precision_score
import torch
from torchvision import models, transforms
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import os
from matplotlib import pylab as P
import PIL.Image
import saliency.core as saliency
from tqdm import tqdm
import cv2
import matplotlib.patches as patches
from scipy.ndimage import zoom

In [ ]:

#Make Labels
df = pd.read_csv("/content/drive/MyDrive/Saliency Map Research 2023/stage_2_train_labels.csv")
df2 = df[['patientId','Target']]
df2 = df2.drop_duplicates()
# Separate positive class (Target = 1) rows from the dataframes
df_positive = df2[(df2['Target'] == 1)]

df_positive = df_positive.reset_index(drop=True)

df_positive = df_positive[df_positive['patientId'] != "1d193b54-7f79-4a6a-bcaf-352a03025019"]

#Path Split
root = '/content/drive/MyDrive/Saliency Map Research 2023/Pneumonia Dataset/stage_2_train_images_png'

# Create a list of positive class image paths
positive_paths = [os.path.join(root, image + '.dcm.jpg.png') for image in df_positive['patientId']]

# Print the combined dataframe with positive class data and the list of paths
print(df_positive)
print(positive_paths)



In [ ]:

# Function to load saliency maps from .pt files and apply normalization
def load_saliency_map(file_path, percentile=99):
    saliency_map = torch.load(file_path)
    #saliency_map = saliency_map.numpy()  # Convert PyTorch tensor to NumPy array

    # Perform the normalization
    #image_2d = np.sum(np.abs(saliency_map), axis=2)
    #vmax = np.percentile(image_2d, percentile)
    #vmin = np.min(image_2d)
    #image_normalized = np.clip((image_2d - vmin) / (vmax - vmin), 0, 1)

    # Convert the normalized 2D array back to PyTorch tensor
    #saliency_map_normalized = torch.tensor(image_normalized, dtype=torch.float32)
    saliency_map_normalized = torch.tensor(saliency_map, dtype=torch.float32)

    return saliency_map_normalized  # Move the tensor to the GPU


In [ ]:
def load_bounding_box_mask(file_path, target_size):
    # Load the bounding box mask array from the file path
    bounding_box_mask = np.load(file_path)

    # Resize the bounding box mask to the target size (same as saliency map size)
    bounding_box_mask_resized = zoom(bounding_box_mask, (target_size[0] / bounding_box_mask.shape[0], target_size[1] / bounding_box_mask.shape[1]), order=0)

    # Convert the bounding box mask to a PyTorch tensor
    bounding_box_mask_tensor = torch.tensor(bounding_box_mask_resized, dtype=torch.float32)

    return bounding_box_mask_tensor

In [ ]:

def auprc(smap, bb_mask):
    # Threshold the saliency map to create a binary mask
    threshold = 0.5  # Adjust this threshold as needed
    binary_smap = (smap >= threshold).float()

    # Flatten the binary saliency map and ground truth mask to 1D arrays
    smap_flat = binary_smap.flatten()
    bb_mask_flat = bb_mask.flatten()

    # Calculate precision and recall values
    precision, recall, _ = precision_recall_curve(bb_mask_flat, smap_flat)

    # Calculate AUPRC using the trapezoidal rule
    auprc_val = auc(recall, precision)

    return auprc_val


In [ ]:
def display_images(saliency_map, ground_truth_mask, mean_auprc):
    # Move the saliency map tensor to CPU
    saliency_map_cpu = saliency_map.cpu()

    # Convert the tensors to numpy arrays
    saliency_map_np = saliency_map_cpu.detach().numpy()
    ground_truth_mask_np = ground_truth_mask

    # Display the saliency map and ground truth mask side by side
    plt.figure(figsize=(12, 6))  # Adjust figure size

    # Display the saliency map
    plt.subplot(1, 2, 1)
    plt.imshow(saliency_map_np, cmap='plasma')
    plt.title('Saliency Map')
    plt.axis('off')  # Remove axes and ticks

    # Display the ground truth mask
    plt.subplot(1, 2, 2)
    plt.imshow(ground_truth_mask_np, cmap='plasma')
    plt.title('Ground Truth Mask')
    plt.axis('off')  # Remove axes and ticks

    # Add the mean AUPRC to the title
    plt.suptitle(f"Mean AUPRC: {mean_auprc:.4f}", fontsize=14, y=1.05)

    plt.show()

In [ ]:
# Define the parent directory containing folders of saliency maps
parent_dir = "/content/drive/MyDrive/Saliency Map Research 2023/SaliencyMaps/Pneumonia Resnet (Tensor)"

# Get a list of subfolders (each folder represents a technique)
technique_folders = [folder for folder in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, folder))]

In [ ]:
technique_folders = ['GradCAM', 'Guided GradCAM', 'GradCAM++', 'ScoreCAM']
#technique_folders = ['XRAI']

In [ ]:
import warnings

target_size = (224, 224)

# Store the results in a list of dictionaries
results_list = []

# List to store patient IDs that caused errors
error_ids = []

# Loop through each technique folder
for technique_folder in technique_folders:
    #if technique_folder == "XRAI":
    #    print(f"Skipping {technique_folder} folder...")
    #    continue  # Skip the "XRAI" folder and move to the next folder

    print(f"Technique Folder: {technique_folder}")

    # Use warnings to suppress the deprecation warning messages
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=DeprecationWarning)

        # Define paths to the folders containing the saliency maps and ground truth masks for this technique
        saliency_maps_folder = os.path.join(parent_dir, technique_folder)
        bbox_masks_folder = "/content/drive/MyDrive/Saliency Map Research 2023/Pneumonia Dataset/Bounding Boxes (arrays)"

        auprc_list = []

       # Loop through patients in the current folder
        for idx, row in tqdm(df_positive.iterrows(), desc='Processing Patients', total=len(df_positive), leave=False):
            label = row['patientId']
            try:
                saliency_map = load_saliency_map(os.path.join(saliency_maps_folder, f"{label}_saliency_map.pt"))
                bbox_path = os.path.join(bbox_masks_folder, label + '_mask.npy')
                bbox_mask = load_bounding_box_mask(bbox_path, target_size)

                auprc_val = auprc(saliency_map, bbox_mask)  # Calculate AUPRC using CPU tensor
                auprc_list.append(auprc_val)
            except Exception as e:
                print(f"Error processing Patient ID: {label}. Exception: {e}")
                error_ids.append(label)
                continue

        # Calculate the mean AUPRC across all labels for this technique
        mean_auprc = np.mean(auprc_list)
        std_auprc = np.std(auprc_list)

        results_folder = '/content/drive/MyDrive/Saliency Map Research 2023/Utility/Results/Pneumonia Dense'

         # Save the mean_auprc result in a text file
        mean_results_file_path = os.path.join(results_folder, f"{technique_folder}_mean_auprc.txt")
        with open(mean_results_file_path, 'w') as file:
            file.write(f"Mean AUPRC: {mean_auprc}")

        # Save the std_auprc result in a text file
        std_results_file_path = os.path.join(results_folder, f"{technique_folder}_std_auprc.txt")
        with open(std_results_file_path, 'w') as file:
            file.write(f"Standard Deviation (SD) AUPRC: {std_auprc}")

        # Store the results in a dictionary
        results_dict = {"Technique Folder": technique_folder, "Mean AUPRC": mean_auprc, "SD AUPRC": std_auprc}
        results_list.append(results_dict)

        # Display an example of the saliency map with the ground truth mask
        example_idx = 0  # Change this index to show a different example if needed
        display_images(saliency_map, bbox_mask, mean_auprc)

        print(f"Mean AUPRC ({technique_folder}):", mean_auprc)
        print(f"Standard Deviation (SD) AUPRC ({technique_folder}):", std_auprc)
        print()  # Add an empty line for better readability between techniques


# Print the patient IDs that caused errors
print("\nPatient IDs with errors:")
print(error_ids)


In [ ]:
display_images(saliency_map, bbox_mask, mean_auprc)